# Predicting House Sale Prices

In this project we will be build a machine learning model based on linear regression method and try to improve it.

Our data is collection of houses features from city Ames, Iowa, United States from 2006 to 2010.The dataset contains 2930 observations and a large number of explanatory variables (23 nominal, 23 ordinal, 14 discrete, and 20 continuous) involved in assessing home values.

Lets start.


In [75]:
#import 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [76]:
#open the data

data = pd.read_csv('AmesHousing.tsv', delimiter = '\t')
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [77]:
#create the required functions

def transform_features(dframe):
    return dframe

def select_features(dframe):
    return dframe[['Gr Liv Area', 'SalePrice']]

def train_and_test(dframe):
    train = dframe[:1460]
    test = dframe[1460:]
    
    train_num = train.select_dtypes(include=['integer', 'float']) 
    test_num = test.select_dtypes(include=['integer', 'float']) 
    features = train_num.columns.drop('SalePrice')
    
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    prediction = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], prediction)
    rmse = np.sqrt(mse)
    return rmse

In [78]:
#make a first step

transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data)

rmse

57088.25161263909

### Featuring Engineering

First and more careful stage is data preparation. On this level we need to clean the data, decide what we'll do with null values, drop them or fill, and chose the right features which can make our model more precise.

For the beginning we find the columns with the null values and drop the columns with more than 5% of missing values.

In [79]:
miss_values = data.isnull().sum()
miss_values.sort_values(ascending=False).head(10)

Pool QC          2917
Misc Feature     2824
Alley            2732
Fence            2358
Fireplace Qu     1422
Lot Frontage      490
Garage Qual       159
Garage Yr Blt     159
Garage Cond       159
Garage Finish     159
dtype: int64

In [80]:
# drop the columns with more than 5% of missing values
columns_with_nulls = miss_values[(miss_values > (data.shape[0] *0.05))]
data = data.drop(columns_with_nulls.index, axis = 1)
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900


In [81]:
#find the other numeric missing values and prepare dictionary for filling

num_cols = data.select_dtypes(include=['integer', 'float']).isnull().sum()
num_cols_index = num_cols[num_cols > 0].index
fill_values = data[num_cols_index].mode().to_dict(orient='records')[0]
fill_values

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [82]:
#fill the missing numeric values
data = data.fillna(fill_values)
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900


select_dtypes(include=['integer', 'float']).isnull().sum()

Then we find the text columns with missing values and drop them all.

In [83]:
#find the text columns with missing values

text_values = data.select_dtypes(include=['object']).isnull().sum()
text_values_index = text_values[text_values!=0].index

In [84]:
#drop them
data = data.drop(text_values_index, axis=1)
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,...,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,120,0,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,...,0,0,0,0,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,3,2010,WD,Normal,189900


In the end drop the columns which havent useful meaning for machine learning and check the date columns.

In [85]:
#drop the useless columns

data = data.drop(['PID', 'Order','Mo Sold', 'Sale Condition', 'Sale Type'], axis=1)

In [86]:
#check the date columms

data['Yr Sold'].value_counts()

2007    694
2009    648
2006    625
2008    622
2010    341
Name: Yr Sold, dtype: int64

In [87]:
data['Year Built'].value_counts().sort_index(ascending=False).head()

2010      3
2009     25
2008     49
2007    109
2006    138
Name: Year Built, dtype: int64

In [88]:
data['Year Remod/Add'].value_counts().sort_index(ascending=False).head()

2010     13
2009     34
2008     81
2007    164
2006    202
Name: Year Remod/Add, dtype: int64

Check the different between years, cause we dont use in our model houses with false information - if houses sold before they was built or remod, their features can't contain useful truly information.

In [89]:
#create the new columns

data['years_before_sales'] = data['Yr Sold'] - data['Year Built']
data['years_since_remod'] = data['Yr Sold'] - data['Year Remod/Add']

In [90]:
data[data['years_before_sales']<0]

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Yr Sold,SalePrice,years_before_sales,years_since_remod
2180,20,RL,39290,Pave,IR1,Bnk,AllPub,Inside,Gtl,Edwards,...,484,0,0,0,0,17000,2007,183850,-1,-2


In [91]:
data[data['years_since_remod']<0]

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Yr Sold,SalePrice,years_before_sales,years_since_remod
1702,60,RL,16659,Pave,IR1,Lvl,AllPub,Corner,Gtl,NridgHt,...,368,0,0,0,0,0,2007,260116,0,-1
2180,20,RL,39290,Pave,IR1,Bnk,AllPub,Inside,Gtl,Edwards,...,484,0,0,0,0,17000,2007,183850,-1,-2
2181,60,RL,40094,Pave,IR1,Bnk,AllPub,Inside,Gtl,Edwards,...,406,0,0,0,0,0,2007,184750,0,-1


In [92]:
#drop this rows
data = data.drop([1702, 2180, 2181], axis=0)

#drop date columns with 'pure' values
data = data.drop(['Yr Sold', 'Year Built', 'Year Remod/Add'], axis=1)

In [93]:
data.head()

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,years_before_sales,years_since_remod
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,210,62,0,0,0,0,0,215000,50,50
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,140,0,0,0,120,0,0,105000,49,49
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,393,36,0,0,0,0,12500,172000,52,52
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,...,0,0,0,0,0,0,0,244000,42,42
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,212,34,0,0,0,0,0,189900,13,12


Now let's check the cleaned and prepared dataset on the ready-made functions.

In [94]:
#insert the current dataset

transform_data = transform_features(data)
filtered_data = select_features(transform_data)
rmse = train_and_test(filtered_data)

rmse

55275.36731241307

### Feature Selection

Now to find the lowest Root Mean Square Error we have to select the appropriate features.

In [98]:
#generate the correlation table

numeric_data = data.select_dtypes(include=['integer', 'float'])
correlation = numeric_data.corr()
correlation

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,years_before_sales,years_since_remod
MS SubClass,1.000000,-0.204638,0.040285,-0.067498,0.005502,-0.058276,-0.070936,-0.129873,-0.219990,-0.250497,...,-0.016283,-0.013309,-0.022963,-0.037988,-0.050696,-0.003451,-0.024356,-0.085128,-0.037655,-0.044886
Lot Area,-0.204638,1.000000,0.090426,-0.033487,0.114982,0.180192,0.084454,0.020899,0.241353,0.321311,...,0.152849,0.089202,0.023004,0.016619,0.056135,0.094421,0.038013,0.267520,-0.020882,-0.019848
Overall Qual,0.040285,0.090426,1.000000,-0.093907,0.418972,0.278274,-0.040551,0.268760,0.545792,0.474888,...,0.253120,0.290553,-0.139907,0.018537,0.042412,0.030638,-0.027685,0.801206,-0.596358,-0.570231
Overall Cond,-0.067498,-0.033487,-0.093907,1.000000,-0.132071,-0.050143,0.040846,-0.136758,-0.174840,-0.157746,...,0.021046,-0.067152,0.071287,0.043804,0.043915,-0.016822,0.047066,-0.101540,0.369267,-0.046373
Mas Vnr Area,0.005502,0.114982,0.418972,-0.132071,1.000000,0.284305,-0.014139,0.087535,0.378545,0.376205,...,0.158743,0.123395,-0.109775,0.014689,0.068366,0.005143,-0.022884,0.506983,-0.306163,-0.190821
BsmtFin SF 1,-0.058276,0.180192,0.278274,-0.050143,0.284305,1.000000,-0.053656,-0.487829,0.522775,0.439662,...,0.216577,0.105248,-0.100343,0.051644,0.098336,0.085531,0.015748,0.439284,-0.278070,-0.148456
BsmtFin SF 2,-0.070936,0.084454,-0.040551,0.040846,-0.014139,-0.053656,1.000000,-0.238304,0.093368,0.088282,...,0.099212,-0.003918,0.032310,-0.023347,0.062897,0.044386,-0.002650,0.006127,0.027250,0.062075
Bsmt Unf SF,-0.129873,0.020899,0.268760,-0.136758,0.087535,-0.487829,-0.238304,1.000000,0.414595,0.296824,...,-0.040532,0.113352,0.006913,-0.005256,-0.047593,-0.031912,-0.026767,0.182751,-0.129199,-0.166312
Total Bsmt SF,-0.219990,0.241353,0.545792,-0.174840,0.378545,0.522775,0.093368,0.414595,1.000000,0.792227,...,0.222595,0.222762,-0.084576,0.039180,0.078435,0.073808,-0.011832,0.644012,-0.409209,-0.298629
1st Flr SF,-0.250497,0.321311,0.474888,-0.157746,0.376205,0.439662,0.088282,0.296824,0.792227,1.000000,...,0.218907,0.212943,-0.065170,0.045636,0.102147,0.124920,-0.011628,0.635185,-0.310902,-0.241824


In [101]:
#view tke correlation with SalePrice column

saleprice_corr = numeric_data.corr()['SalePrice'].abs().sort_values(ascending=False)
saleprice_corr

SalePrice             1.000000
Overall Qual          0.801206
Gr Liv Area           0.717596
Garage Cars           0.648361
Total Bsmt SF         0.644012
Garage Area           0.641425
1st Flr SF            0.635185
years_before_sales    0.558979
Full Bath             0.546118
years_since_remod     0.534985
Mas Vnr Area          0.506983
TotRms AbvGrd         0.498574
Fireplaces            0.474831
BsmtFin SF 1          0.439284
Wood Deck SF          0.328183
Open Porch SF         0.316262
Half Bath             0.284871
Bsmt Full Bath        0.276258
2nd Flr SF            0.269601
Lot Area              0.267520
Bsmt Unf SF           0.182751
Bedroom AbvGr         0.143916
Enclosed Porch        0.128685
Kitchen AbvGr         0.119760
Screen Porch          0.112280
Overall Cond          0.101540
MS SubClass           0.085128
Pool Area             0.068438
Low Qual Fin SF       0.037629
Bsmt Half Bath        0.035875
3Ssn Porch            0.032268
Misc Val              0.019273
BsmtFin 

### COntinue